# Regresor logístico

Notebook escrito por Multivacs en 2025.  
Como referencia [logistic.ipynb](https://colab.research.google.com/github/jaspock/me/blob/main/docs/materials/transformers/assets/notebooks/logistic.ipynb) de Juan Antonio Pérez.

Este notebook es parte de la serie [Introducción a las Redes Neuronales](https://multivacs.com/tags/intro-nn/).



## Instalación de dependencias

Instalamos las librerías necesarias para ejecutar el código

In [ ]:
%%capture
%pip install matplotlib numpy torch scikit-learn

## Establecemos semilla

Configuramos una semilla inicial para hacer que los resultados sean reproducibles. Esto es para todas las funciones que utilicen métodos aleatorios, tratar de obtener los mismos resultados independientemente de la ejecución.

Digo tratar ya que aún así, es posible que torch utilice métodos no deterministas para la multiplicación de matrices, por lo que no se garantiza la reproducibilidad, aunque serán muy parecidos.

In [ ]:
import os
# establecemos esta variable de entorno previamente a importar pytorch para evitar operaciones no deterministas en GPU
os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8"

import random
import numpy as np
import torch

def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.use_deterministic_algorithms(True)

set_seed(42)

## Generamos datos sintéticos

Vamos a generar un dataset de manera aleatoria un vector de dos elementos por cada clase, utilizando una distribución normal.

El siguiente código genera datos para las dos clases utilizando `scikit-learn` y su función `sklearn.datasets.make_blobs`.  
Esta función devuelve una tupla con los datos generados y la correspondiente etiqueta (en nuestro caso, 0 y 1).

In [ ]:
from sklearn.datasets import make_blobs
import matplotlib.pyplot as plt

class_centers = [[1,2], [3,4]]
samples = 100  # número de muestras por clase
features = 2  # dimensión de las muestras (num columnas)

Xn, yn = make_blobs(n_samples=samples, centers=class_centers, cluster_std=1, n_features=features, random_state=42)

print(f"Xn.shape = {Xn.shape}")
print(f"yn.shape = {yn.shape}")
print(f"Xn[:3] = {Xn[:3]}")
print(f"yn[:3] = {yn[:3]}")

plt.scatter(Xn[:,0], Xn[:,1], c=yn, cmap='tab20c')

## Inicializar tensores

Como sabrás, en cuanto a computación, las CPUs están diseñadas para uso general, llevando a cabo tareas diversas y normalmente, de manera secuencial (en fila).  
Mientras que las GPUs son hardware dedicado a una tarea muy específica, como es el procesamiento y cálculo en paralelo, haciéndolos muy eficientes para realizar tareas simultáneas como el entrenamiento de modelos de deep learning.

Por tanto, vamos a pasar de vectores NumPy (CPU-efficient) a tensores PyTorch (GPU-optimized).  
Para ello, utilizaremos la función nativa de PyTorch `torch.from_numpy`, que recoge como input un vector NumPy y lo transforma a un objeto de tipo `torch.Tensor`.

Como dato adicional, NumPy utiliza doble precisión (FP64) por defecto, mientras que PyTorch usa 32 bits, ya que las GPUs son más rápidas en precisión simple y la diferencia en precisión es normalmente insignificante. Por eso, convertimos primero el vector NumPy de 64 a 32 bits.

In [ ]:
import numpy as np
import torch

device = "cuda:0" if torch.cuda.is_available() else "cpu"  # comprobamos si tenemos gpu disponible
print(f'Usando {device}')

X = torch.from_numpy(np.float32(Xn)).to(device)
y = torch.from_numpy(np.float32(yn)).to(device)

print(f"type(X) = {type(X)}, type(y) = {type(y)}")
print(f"X[:3] = {X[:3]}")
print(f"y[:3] = {y[:3]}")

## Dividir los datos en entrenamiento y test

En Machine Learning solemos diferenciar entre el conjunto de datos que utilizaremos para entrenar el modelo, y aquél usado para comprobar su rendimiento.  
Esto es para poder ver cuál es la capacidad de generalización de nuestro modelo, ya que lo interesante no es que el modelo se aprenda todas las respuestas de nuestro listado de memoria (overfitting), sino que sea capaz de dada una entrada que previamente no ha visto, poder inferir la probabilidad de pertenencia a una clase.

Adicionalmente, se crea un tercer subconjunto de los datos dedicado a validación. Para entender su función, digamos que es como una manera de ir comprobando periódicamente cada x iteraciones, si el modelo ha sido capaz de mejorar o no respecto a las iteraciones previas. Esto es para tratar de forzar que el modelo pare cuando haya dejado de mejorar en su entrenamiento y evitar el sobreentrenamiento.

Para hacer la división crearemos una máscara poniendo a 0 aquellas muestras dedidacadas a entreno y a 1 las de test.

In [ ]:
mask = torch.ones(X.shape[0], dtype=bool).to(device)
mask[::3] = 0  # cada 3 elementos lo utilizamos como test
X_train, y_train = X[mask], y[mask]
X_test, y_test = X[torch.logical_not(mask)], y[torch.logical_not(mask)]

print(f"X_train.shape = {X_train.shape}")
print(f"y_train.shape = {y_train.shape}")
print(f"X_test.shape = {X_test.shape}")
print(f"y_test.shape = {y_test.shape}")

## Funciones auxiliares

Definimos las siguientes funciones que serán utilizadas durante el entrenamiento e inferencia del modelo:

- `regressor`: inicializa y devuelve los parámetros aprendibles (pesos y sesgo). Hay varias estrategias para inicializar estos parámetros, por ahora lo iniciaremos de manera aleatoria.
- `sigmoid`: devuelve un tensor en el que cada elemento es la sigmoide del elemento correspondiente del tensor de entrada.
- `forward`: devuelve la salida del regresor logístico.
- `binary_cross_entropy`: devuelve la función de pérdida de entropía cruzada, dado el vector de predicciones y la correspondiente salida esperada.

In [ ]:
def regressor(size):
    weights = torch.rand(size, dtype=torch.float32).to(device)
    bias = torch.rand(1, dtype=torch.float32).to(device)
    return weights, bias

def sigmoid(x):
    return 1 / (1 + torch.exp(-x))

def forward(X, weights, bias):
    return sigmoid(torch.matmul(X,weights) + bias)

def binary_cross_entropy(y_truth, y_pred):
    m = 1 / y_truth.shape[0]  # y_truth.shape[0] es el tamaño del mini-batch
    return -m * (y_truth * torch.log(y_pred) +
                    (1 - y_truth) * torch.log(1 - y_pred)).sum()

## Entrenando el modelo

Estamos listos para entrenar nuestro modelo para hacer clasificación de dos clases. El proceso de entrenamiento lo establecemos en la función `train`, que recibe como entrada el mini-batch de datos, la salida esperada, los parámetros de peso y bias, el learning rate, número de pasos a realizar, y el número de pasos en los que se evalúa el modelo para propósitos de logging.

La función devuelve el vector de pesos y bias aprendidos, y el error tras el último paso de entrenamiento.

In [ ]:
def loss_backward(y_truth, y_pred, X):
    '''
    Calcula el gradiente.
    '''
    err = (y_pred - y_truth)
    grad_w = (1 / y_truth.shape[0]) * torch.matmul(err, X)
    grad_b = (1 / y_truth.shape[0]) * torch.sum(err)
    return grad_w, grad_b

def optimizer_step(weights, bias, grad_w, grad_b, lr=0.01):
    '''
    Actualiza los parámetros del modelo.
    '''
    weights = weights - lr * grad_w
    bias = bias - lr * grad_b
    return weights, bias

def train(X, y_truth, weights, bias, lr=0.01, training_steps=1000, valid_steps=100):
    for i in range(training_steps):
        y_pred = forward(X, weights, bias)
        grad_w, grad_b = loss_backward(y_truth, y_pred, X)
        weights, bias = optimizer_step(weights, bias, grad_w, grad_b, lr)
        if i % valid_steps == 0:
            loss = binary_cross_entropy(y_truth, y_pred).item()  
            # item() devuelve un escalar a partir de un tensor de un único valor 
            print (f'Step [{i}/{training_steps}], loss: {loss:.2f}')
    loss = binary_cross_entropy(y_truth, y_pred).item()
    print (f'Step [{training_steps}/{training_steps}], loss: {loss:.2f}')
    return weights, bias, loss

In [ ]:
learning_rate = 0.05
training_steps = 100
valid_steps = 10

weights, bias = regressor(X_train.shape[1])
weights, bias, bn_train = train(X_train, y_train, weights, bias,
                            lr=learning_rate, training_steps=training_steps, valid_steps=valid_steps)

## Evaluar el modelo (test)

Una vez tenemos el modelo entrenado, con sus pesos y bias, usamos el conjunto de test para medir su rendimiento

In [ ]:
print(f'Learned logistic regressor: y = σ({weights[0]:.2f}*x1 + {weights[1]:.2f}*x2 + {bias.item():.2f})')
y_pred = forward(X_test, weights, bias)
loss = binary_cross_entropy(y_test, y_pred).item()
print(f'Binary cross-entropy on the test set: {loss:.2f}')
prediction = y_pred > 0.5 
correct = prediction == y_test
accuracy = (torch.sum(correct) / y_test.shape[0])*100
print (f'Accuracy on the test set: {accuracy:.2f}%')

## Visualizamos la frontera de decisión

Por último dibujamos en una gráfica utilizando `matplotlib` la frontera de decisión del modelo, es decir, la línea (hiperplano) divisoria que separa las dos clases.

In [ ]:
r = -weights[0].item() / weights[1].item()  # slope
t = -bias.item() / weights[1].item()  # intercept

plt.scatter(Xn[:,0], Xn[:,1], c=yn, cmap='tab20c')
plt.title(f"Hyperplane learned by the logistic regressor")
plt.xlabel("x")
plt.ylabel("y")
x_hyperplane = np.linspace(0,6,100)
y_hyperplane = r*x_hyperplane+t
plt.plot(x_hyperplane, y_hyperplane, '-c')  # -c means solid cyan line
plt.show()